### **Reading Raw Ingested Date from Bronze layer**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *


In [0]:
df_products = spark.read.format("parquet")\
                   .load('abfss://bronze@stretailenvdev.dfs.core.windows.net/f_Products')

In [0]:
df_products.count()

### **Tranformation**

In [0]:
df_products.printSchema()

In [0]:
df_products = (
    df_products.dropDuplicates(["ProductID"])
    .fillna({
        "ProductName": "Unknown",
        "Price": 0.0,
    })
    .filter(col("Price") >= 0)  # remove invalid price
)

In [0]:
df_products.display()

### **Writing Data in Silver layer**

In [0]:
df_products.write.format("delta")\
           .mode("overwrite")\
            .save("abfss://silver@stretailenvdev.dfs.core.windows.net/s_Products")